In [ ]:
!pip install valorant

In [ ]:
import os
import valorant

In [ ]:
os.environ["VALPY-KEY"] = "RGAPI-88d6f215-b22d-4c82-ba97-d73924fc6db4"

In [ ]:
KEY = os.environ["VALPY-KEY"]
client = valorant.Client(KEY, locale=None)

In [ ]:
client.get_user_by_name(name="viikset#BR1", route="americas")

In [ ]:
client.get_match(id="c229a683-a205-4543-a174-522956e8b72d")

In [ ]:
client.get_session()

In [ ]:
token = 'RGAPI-68e255f5-6d59-4152-b84b-35f67be5e3fe'

In [ ]:
puuid = 'puuid=t6hgE7vM78zeB7VZqIF9BV4VC3xqSFyBQ_QaAK_l2xRSfxTqaTmh0Adqwc7V7QamAPszSWlTyDf2lg'

In [ ]:
import requests
response = requests.get('https://br.api.riotgames.com/lol/summoner/v4/summoners/by-name/viikset', headers={'Authorization': f'Bearer token {token}'})

In [ ]:
response.json()

# Main

In [1]:
import os
import json
import time
import requests
import pandas as pd

from datetime import datetime

In [2]:
date_time = datetime.now()
                    
if time.tzname[0] == 'UTC':
    date_time = date_time - timedelta(hours=3)

In [3]:
datetime = date_time.strftime("%Y%m%dT%H:%M:%S")

In [4]:
timestamp = int(date_time.timestamp())

In [5]:
formated_datetime = date_time.strftime("%Y%m%d_%Hh%Mm%Ss")

In [6]:
users = {
    "viikset": "BR1",
    "TADALA": "TADAL",
    "DIGAOTJS": "BR1",
    "mBirth": "BR1",
    "Kubata": "4881"
}

In [7]:
puuids = {
    "viikset": "f2f7c867-27ba-5291-a23c-bbdcfc079e76",
    "TADALA": "35a2368e-586f-53ec-8ee8-ea9ebdca7d6f",
    "DIGAOTJS": "2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663",
    "mBirth": "6c241ab0-1c2f-5d5b-8908-4e809df8f6ca",
    "Kubata": "b2310858-1741-5d6f-8499-cb1f85bd182a"
}

## Get Account Data

In [8]:
all_match_ids = []

df_account = pd.DataFrame()
df_mmr_data = pd.DataFrame()
df_mmr_history = pd.DataFrame()
df_matchs = pd.DataFrame()


for name, tag in users.items():
    
    print(f'Get data for {name}...')
    
    # Accounts Data
    print(f'Get account data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}")
    
    with open(f'account/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_account = pd.concat([df_account, pd.json_normalize(response.json()['data'])])
        
        
    # MMR Data
    print(f'Get MMR data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr/na/{name}/{tag}")
    
    with open(f'mmr_data/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_mmr_data = pd.concat([df_mmr_data, pd.json_normalize(response.json()['data'])])  
        
    # MMR History
    print(f'Get MMR History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr-history/na/{name}/{tag}")
    
    with open(f'mmr_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    temp_mmr_history = pd.json_normalize(response.json()['data'])
    temp_mmr_history['name'] = response.json()['name']
    temp_mmr_history['tag'] = response.json()['tag']
    
    df_mmr_history = pd.concat([df_mmr_history, temp_mmr_history])
    
    # Match History
    print(f'Get Match History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/matches/na/{name}/{tag}?filter=competitive")
    
    with open(f'match_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    # Listing All Match IDs
    metadata = []
    for i in response.json()['data']:
        all_match_ids.append(i['metadata']['matchid'])
        
        metadata.append(i['metadata'])
        
    df_matchs = pd.concat([df_matchs, pd.DataFrame(metadata)])
    
    print('\n')

Get data for viikset...
Get account data for viikset...
Get MMR data for viikset...
Get MMR History data for viikset...
Get Match History data for viikset...


Get data for TADALA...
Get account data for TADALA...
Get MMR data for TADALA...
Get MMR History data for TADALA...
Get Match History data for TADALA...


Get data for DIGAOTJS...
Get account data for DIGAOTJS...
Get MMR data for DIGAOTJS...
Get MMR History data for DIGAOTJS...
Get Match History data for DIGAOTJS...


Get data for mBirth...
Get account data for mBirth...
Get MMR data for mBirth...
Get MMR History data for mBirth...
Get Match History data for mBirth...


Get data for Kubata...
Get account data for Kubata...
Get MMR data for Kubata...
Get MMR History data for Kubata...
Get Match History data for Kubata...




# Get All Match History

In [15]:
all_match_ids = list(set(all_match_ids))

In [42]:
metadata = []
rounds = []
kills = []

df_players = pd.DataFrame()
df_teams = pd.DataFrame()

for matchid in all_match_ids:
    
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/match/{matchid}")
    
    with open(f'match/{matchid}.json', 'w') as f:
        json.dump(response.json(), f)
        
    match = response.json()['data']
        
    # Match Metadata    
    metadata.append(match['metadata'])
    
    # Match Players
    temp_players = pd.json_normalize(match['players']['all_players'])
    temp_players['matchid'] = matchid
    df_players = pd.concat([df_players, temp_players])
    
    # Match Teams
    temp_teams = pd.json_normalize(match['teams'])
    temp_teams['matchid'] = matchid
    df_teams = pd.concat([df_teams, temp_teams])
    
    
    rounds.append(match['rounds'])
    
    
    kills.append(match['kills'])
        
    break
    
df_metadata = pd.DataFrame(metadata)



In [123]:
pd.json_normalize(match['rounds'])

,winning_team,end_type,bomb_planted,bomb_defused,player_stats,plant_events.plant_location.x,plant_events.plant_location.y,plant_events.planted_by.puuid,plant_events.planted_by.display_name,plant_events.planted_by.team,plant_events.plant_site,plant_events.plant_time_in_round,plant_events.player_locations_on_plant,defuse_events.defuse_location,defuse_events.defused_by,defuse_events.defuse_time_in_round,defuse_events.player_locations_on_defuse,defuse_events.defuse_location.x,defuse_events.defuse_location.y,defuse_events.defused_by.puuid,defuse_events.defused_by.display_name,defuse_events.defused_by.team,plant_events.plant_location,plant_events.planted_by
0,Red,Bomb detonated,True,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",7916.0,4174.0,c512b1c8-6264-5196-a39c-f755e8b3b8c4,VznK#1603,Red,A,75984.0,[{'player_puuid': 'b4127d4b-e7ff-5f39-b838-468...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Red,Eliminated,True,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",7514.0,3937.0,f2f7c867-27ba-5291-a23c-bbdcfc079e76,viikset#BR1,Red,A,44285.0,[{'player_puuid': 'e9331dd4-2ed8-572d-97e5-11a...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Red,Eliminated,True,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",8166.0,-5292.0,f2f7c867-27ba-5291-a23c-bbdcfc079e76,viikset#BR1,Red,B,74030.0,[{'player_puuid': '6c241ab0-1c2f-5d5b-8908-4e8...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Blue,Bomb defused,True,True,"[{'ability_casts': {'c_casts': None, 'q_casts'...",7982.0,4030.0,c512b1c8-6264-5196-a39c-f755e8b3b8c4,VznK#1603,Red,A,67017.0,[{'player_puuid': 'b4127d4b-e7ff-5f39-b838-468...,NaN,NaN,90810.0,[{'player_puuid': 'f4daf6b0-cdad-5484-953a-c93...,7966.0,4010.0,47b6f257-3991-547f-a138-3b146f35aa1f,j3d#trev,Blue,NaN,NaN
4,Red,Eliminated,True,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",8064.0,-6094.0,f2f7c867-27ba-5291-a23c-bbdcfc079e76,viikset#BR1,Red,B,39294.0,[{'player_puuid': '6c241ab0-1c2f-5d5b-8908-4e8...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Blue,Bomb defused,True,True,"[{'ability_casts': {'c_casts': None, 'q_casts'...",8909.0,3568.0,2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663,DIGAOTJS#BR1,Red,A,80487.0,[{'player_puuid': '35a2368e-586f-53ec-8ee8-ea9...,NaN,NaN,122527.0,[{'player_puuid': 'e9331dd4-2ed8-572d-97e5-11a...,8734.0,3666.0,e9331dd4-2ed8-572d-97e5-11ac0925220f,ryanzin FZN#4605,Blue,NaN,NaN
6,Blue,Eliminated,False,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Red,Bomb detonated,True,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",7658.0,-6326.0,f2f7c867-27ba-5291-a23c-bbdcfc079e76,viikset#BR1,Red,B,38976.0,[{'player_puuid': 'b4127d4b-e7ff-5f39-b838-468...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Blue,Bomb defused,True,True,"[{'ability_casts': {'c_casts': None, 'q_casts'...",7766.0,4112.0,35a2368e-586f-53ec-8ee8-ea9ebdca7d6f,TADALA#TADAL,Red,A,99503.0,[{'player_puuid': '35a2368e-586f-53ec-8ee8-ea9...,NaN,NaN,115953.0,[{'player_puuid': '47b6f257-3991-547f-a138-3b1...,7749.0,4154.0,47b6f257-3991-547f-a138-3b146f35aa1f,j3d#trev,Blue,NaN,NaN
9,Blue,Eliminated,False,False,"[{'ability_casts': {'c_casts': None, 'q_casts'...",NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
count = 0
for player in players:
    print(count)
    aa = pd.json_normalize(player['all_players'])
    break
    

0


In [108]:
pd.set_option('display.max_columns', 100)

In [112]:
aa.head()

,puuid,name,tag,team,level,character,currenttier,currenttier_patched,player_card,player_title,party_id,damage_made,damage_received,session_playtime.minutes,session_playtime.seconds,session_playtime.milliseconds,behavior.afk_rounds,behavior.friendly_fire.incoming,behavior.friendly_fire.outgoing,behavior.rounds_in_spawn,platform.type,platform.os.name,platform.os.version,ability_casts.c_cast,ability_casts.q_cast,ability_casts.e_cast,ability_casts.x_cast,assets.card.small,assets.card.large,assets.card.wide,assets.agent.small,assets.agent.bust,assets.agent.full,assets.agent.killfeed,stats.score,stats.kills,stats.deaths,stats.assists,stats.bodyshots,stats.headshots,stats.legshots,economy.spent.overall,economy.spent.average,economy.loadout_value.overall,economy.loadout_value.average
0,b4127d4b-e7ff-5f39-b838-4688a48e1659,Aline,ninne,Blue,73,Sage,10,Silver 2,3d630ad7-4d72-08a4-4a9d-30af16783d06,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,2214,3029,391,23460,23460000,0,0,0,0,PC,Windows,10.0.22621.1.256.64bit,20,24,10,3,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/agents/569fdd95...,https://media.valorant-api.com/agents/569fdd95...,https://media.valorant-api.com/agents/569fdd95...,https://media.valorant-api.com/agents/569fdd95...,3960,14,17,12,48,4,11,60500,2750,71000,3227
1,2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663,DIGAOTJS,BR1,Red,77,Breach,9,Silver 1,514ea59a-448c-ba70-9ebb-5faff372b6e2,631f4283-48b1-1855-d646-5e8f80e29821,7634401f-e642-4547-bea2-82b5f38554aa,2957,3382,146,8760,8760000,0,0,0,0,PC,Windows,10.0.19044.1.768.64bit,6,19,22,4,https://media.valorant-api.com/playercards/514...,https://media.valorant-api.com/playercards/514...,https://media.valorant-api.com/playercards/514...,https://media.valorant-api.com/agents/5f8d3a7f...,https://media.valorant-api.com/agents/5f8d3a7f...,https://media.valorant-api.com/agents/5f8d3a7f...,https://media.valorant-api.com/agents/5f8d3a7f...,4993,18,17,4,61,4,3,54200,2464,80650,3666
2,f4daf6b0-cdad-5484-953a-c93c3faf0f26,Digui FZN,2003,Blue,61,Neon,15,Platinum 1,04ca7dfa-48f0-0d6e-587d-8b9ecc78c5e8,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,5113,3567,197,11820,11820000,0,0,0,0,PC,Windows,10.0.19045.1.768.64bit,3,12,21,4,https://media.valorant-api.com/playercards/04c...,https://media.valorant-api.com/playercards/04c...,https://media.valorant-api.com/playercards/04c...,https://media.valorant-api.com/agents/bb2a4828...,https://media.valorant-api.com/agents/bb2a4828...,https://media.valorant-api.com/agents/bb2a4828...,https://media.valorant-api.com/agents/bb2a4828...,8370,27,19,5,92,24,6,54900,2495,80700,3668
3,012591c1-0832-5773-9f29-2ec025187ab6,Zunnie,Zack,Blue,56,Cypher,8,Bronze 3,3d630ad7-4d72-08a4-4a9d-30af16783d06,af20bdd6-4829-2dee-1bd3-77a34b6e2e9e,d0650980-7ca8-4c57-ac0c-df1a8a44a133,3059,1950,61,3660,3660000,0,0,0,0,PC,Windows,10.0.19045.1.256.64bit,39,42,20,3,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/playercards/3d6...,https://media.valorant-api.com/agents/117ed9e3...,https://media.valorant-api.com/agents/117ed9e3...,https://media.valorant-api.com/agents/117ed9e3...,https://media.valorant-api.com/agents/117ed9e3...,4508,15,11,7,45,11,5,56300,2559,65700,2986
4,e9331dd4-2ed8-572d-97e5-11ac0925220f,ryanzin FZN,4605,Blue,106,Viper,12,Gold 1,4b00c963-4ff3-d147-f8ab-92b31a88c8a2,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,3250,2935,162,9720,9720000,0,0,0,0,PC,Windows,10.0.19045.1.768.64bit,21,12,21,1,https://media.valorant-api.com/playercards/4b0...,https://media.valorant-api.com/playercards/4b0...,https://media.valorant-api.com/playercards/4b0...,https://media.valorant-api.com/agents/707eab51...,https://media.valorant-api.com/agents/707eab51...,https://media.valorant-api.com/agents/707ea

In [100]:
pd.DataFrame(player_row['all_players'])

,puuid,name,tag,team,level,character,currenttier,currenttier_patched,player_card,player_title,party_id,session_playtime,behavior,platform,ability_casts,assets,stats,economy,damage_made,damage_received
0,b4127d4b-e7ff-5f39-b838-4688a48e1659,Aline,ninne,Blue,73,Sage,10,Silver 2,3d630ad7-4d72-08a4-4a9d-30af16783d06,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 391, 'seconds': 23460, 'millisecon...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 20, 'q_cast': 24, 'e_cast': 10, 'x_...",{'card': {'small': 'https://media.valorant-api...,"{'score': 3960, 'kills': 14, 'deaths': 17, 'as...","{'spent': {'overall': 60500, 'average': 2750},...",2214,3029
1,2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663,DIGAOTJS,BR1,Red,77,Breach,9,Silver 1,514ea59a-448c-ba70-9ebb-5faff372b6e2,631f4283-48b1-1855-d646-5e8f80e29821,7634401f-e642-4547-bea2-82b5f38554aa,"{'minutes': 146, 'seconds': 8760, 'millisecond...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 6, 'q_cast': 19, 'e_cast': 22, 'x_c...",{'card': {'small': 'https://media.valorant-api...,"{'score': 4993, 'kills': 18, 'deaths': 17, 'as...","{'spent': {'overall': 54200, 'average': 2464},...",2957,3382
2,f4daf6b0-cdad-5484-953a-c93c3faf0f26,Digui FZN,2003,Blue,61,Neon,15,Platinum 1,04ca7dfa-48f0-0d6e-587d-8b9ecc78c5e8,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 197, 'seconds': 11820, 'millisecon...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 3, 'q_cast': 12, 'e_cast': 21, 'x_c...",{'card': {'small': 'https://media.valorant-api...,"{'score': 8370, 'kills': 27, 'deaths': 19, 'as...","{'spent': {'overall': 54900, 'average': 2495},...",5113,3567
3,012591c1-0832-5773-9f29-2ec025187ab6,Zunnie,Zack,Blue,56,Cypher,8,Bronze 3,3d630ad7-4d72-08a4-4a9d-30af16783d06,af20bdd6-4829-2dee-1bd3-77a34b6e2e9e,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 61, 'seconds': 3660, 'milliseconds...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 39, 'q_cast': 42, 'e_cast': 20, 'x_...",{'card': {'small': 'https://media.valorant-api...,"{'score': 4508, 'kills': 15, 'deaths': 11, 'as...","{'spent': {'overall': 56300, 'average': 2559},...",3059,1950
4,e9331dd4-2ed8-572d-97e5-11ac0925220f,ryanzin FZN,4605,Blue,106,Viper,12,Gold 1,4b00c963-4ff3-d147-f8ab-92b31a88c8a2,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 162, 'seconds': 9720, 'millisecond...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 21, 'q_cast': 12, 'e_cast': 21, 'x_...",{'card': {'small': 'https://media.valorant-api...,"{'score': 4884, 'kills': 18, 'deaths': 17, 'as...","{'spent': {'overall': 49950, 'average': 2270},...",3250,2935
5,47b6f257-3991-547f-a138-3b146f35aa1f,j3d,trev,Blue,54,Breach,9,Silver 1,d3c65faf-49c2-9861-cccd-f3b5428c64da,dd9b86b1-4661-1c98-65ac-c09b70a88e74,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 188, 'seconds': 11280, 'millisecon...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 7, 'q_cast': 18, 'e_cast': 32, 'x_c...",{'card': {'small': 'https://media.valorant-api...,"{'score': 3468, 'kills': 12, 'deaths': 16, 'as...","{'spent': {'overall': 52600, 'average': 2391},...",2652,3042
6,35a2368e-586f-53ec-8ee8-ea9ebdca7d6f,TADALA,TADAL,Red,145,Harbor,9,Silver 1,867e85c2-4f4e-797b-7c94-a0b403fe22f3,42e2f97d-4e69-274c-589c-0fbdeb397df9,7634401f-e642-4547-bea2-82b5f38554aa,"{'minutes': 223, 'seconds': 13380, 'millisecon...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 22, 'q_cast': 5, 'e_cast': 15, 'x_c...",{'card': {'small': 'https://media.valo

In [84]:
player['session_playtime']

{'minutes': 391, 'seconds': 23460, 'milliseconds': 23460000}

In [85]:
player['behavior']

{'afk_rounds': 0,
 'friendly_fire': {'incoming': 0, 'outgoing': 0},
 'rounds_in_spawn': 0}

In [86]:
player['platform']

{'type': 'PC', 'os': {'name': 'Windows', 'version': '10.0.22621.1.256.64bit'}}

In [87]:
player['ability_casts']

{'c_cast': 20, 'q_cast': 24, 'e_cast': 10, 'x_cast': 3}

In [90]:
player['assets']

{'card': {'small': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/smallart.png',
  'large': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/largeart.png',
  'wide': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/wideart.png'},
 'agent': {'small': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/displayicon.png',
  'bust': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/bustportrait.png',
  'full': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/fullportrait.png',
  'killfeed': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/killfeedportrait.png'}}

In [91]:
player['assets']['card']

{'small': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/smallart.png',
 'large': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/largeart.png',
 'wide': 'https://media.valorant-api.com/playercards/3d630ad7-4d72-08a4-4a9d-30af16783d06/wideart.png'}

In [92]:
player['assets']['agent']

{'small': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/displayicon.png',
 'bust': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/bustportrait.png',
 'full': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/fullportrait.png',
 'killfeed': 'https://media.valorant-api.com/agents/569fdd95-4d10-43ab-ca70-79becc718b46/killfeedportrait.png'}

In [93]:
player['stats']

{'score': 3960,
 'kills': 14,
 'deaths': 17,
 'assists': 12,
 'bodyshots': 48,
 'headshots': 4,
 'legshots': 11}

In [94]:
player['economy']

{'spent': {'overall': 60500, 'average': 2750},
 'loadout_value': {'overall': 71000, 'average': 3227}}

In [95]:
player['economy']['spent']

{'overall': 60500, 'average': 2750}

In [96]:
player['economy']['loadout_value']

{'overall': 71000, 'average': 3227}

In [77]:
pd.DataFrame(players[0]['all_players']).head(1)

,puuid,name,tag,team,level,character,currenttier,currenttier_patched,player_card,player_title,party_id,session_playtime,behavior,platform,ability_casts,assets,stats,economy,damage_made,damage_received
0,b4127d4b-e7ff-5f39-b838-4688a48e1659,Aline,ninne,Blue,73,Sage,10,Silver 2,3d630ad7-4d72-08a4-4a9d-30af16783d06,0053beda-44f4-47f0-3ec5-68a20f7a0505,d0650980-7ca8-4c57-ac0c-df1a8a44a133,"{'minutes': 391, 'seconds': 23460, 'millisecon...","{'afk_rounds': 0, 'friendly_fire': {'incoming'...","{'type': 'PC', 'os': {'name': 'Windows', 'vers...","{'c_cast': 20, 'q_cast': 24, 'e_cast': 10, 'x_...",{'card': {'small': 'https://media.valorant-api...,"{'score': 3960, 'kills': 14, 'deaths': 17, 'as...","{'spent': {'overall': 60500, 'average': 2750},...",2214,3029


In [75]:
players[0].keys()

dict_keys(['all_players', 'red', 'blue'])